## Data Segmentation into Overlapping Windows

In [ ]:
import pandas as pd

# Load the dataset (adjust the file path if necessary, or skip this if df is already in memory)
df = pd.read_csv('data/data_imputed.csv')
print("Loaded dataset with shape:", df.shape)

# Sort by time within each user group for proper temporal ordering
df = df.sort_values(['user', 'seconds_elapsed']).reset_index(drop=True)

# Define window parameters
window_size = 200          # 2-second window (assuming 100 Hz data -> 200 samples)
step_size = window_size // 2   # 50% overlap -> 100 samples shift

# Segment into windows
X_windows = []   # list to hold window feature arrays
y_windows = []   # list to hold window labels
for user_id, group in df.groupby('user'):
    data = group.drop(columns=['user', 'seconds_elapsed']).values  # drop label and time, use feature values
    n = len(data)
    if n < window_size:
        continue  # skip this user if not enough data for one window (unlikely here)
    # Slide window with 50% overlap
    for start in range(0, n - window_size + 1, step_size):
        end = start + window_size
        X_windows.append(data[start:end])
        y_windows.append(user_id)

# Convert lists to arrays
X_windows = np.array(X_windows)
y_windows = np.array(y_windows)
print("Total windows formed:", X_windows.shape[0])
print("Window shape (timesteps x features):", X_windows.shape[1], "x", X_windows.shape[2])

## Preparing Train, Validation, and Test Sets

In [ ]:
import numpy as np
# restore deprecated alias for legacy packages and python 3.8.8
if not hasattr(np, "int"):
    np.int = int        # or np.int64 if you prefer fixed width

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode the user labels (if they're not already numeric) to integers 0,1,2
encoder = LabelEncoder()
y_enc = encoder.fit_transform(y_windows)  # e.g. "User_1","User_2","User_3" -> 0,1,2

# Stratified split into train+val and test
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_windows, y_enc, test_size=0.20, stratify=y_enc, random_state=42)
# Further split train_val into actual train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, stratify=y_train_val, random_state=42)
    # 0.25 of 0.8 is 0.2, so overall: 60% train, 20% val, 20% test

# Print shapes and class distributions for verification
print("Train set:", X_train.shape, "Validation set:", X_val.shape, "Test set:", X_test.shape)
print("Train class distribution:", np.bincount(y_train))
print("Test class distribution:", np.bincount(y_test))

## Baseline TCN Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_tcn_model(input_shape, num_classes, num_filters=32, kernel_size=3, dilations=[1, 2, 4], dropout_rate=0.1):
    """
    Builds a Temporal Convolutional Network model.
    - input_shape: (timesteps, features)
    - num_classes: number of output classes
    - num_filters: number of convolutional filters in each conv layer
    - kernel_size: size of the 1D convolution kernel
    - dilations: list of dilation rates for each TCN block
    - dropout_rate: dropout rate for SpatialDropout1D layers
    """
    inputs = layers.Input(shape=input_shape)
    x = inputs

    # Optional initial projection to num_filters if input feature dimension is different
    if num_filters != input_shape[-1]:
        x = layers.Conv1D(num_filters, kernel_size=1, padding='same')(x)

    # TCN residual blocks
    for d in dilations:
        residual = x  # save input for residual connection
        # 1st convolution in the block
        x = layers.Conv1D(num_filters, kernel_size, dilation_rate=d, padding='causal')(x)
        x = layers.Activation('relu')(x)
        x = layers.SpatialDropout1D(dropout_rate)(x)
        # 2nd convolution in the block
        x = layers.Conv1D(num_filters, kernel_size, dilation_rate=d, padding='causal')(x)
        x = layers.Activation('relu')(x)
        x = layers.SpatialDropout1D(dropout_rate)(x)
        # Residual connection: make channel dims equal before adding, if needed
        if residual.shape[-1] != x.shape[-1]:
            residual = layers.Conv1D(num_filters, kernel_size=1, padding='same')(residual)
        x = layers.Add()([x, residual])
        x = layers.Activation('relu')(x)  # activation after adding residual

    # Output layer: use the last time step's features for classification
    # (Because of causal convolutions, the last time step contains info from the entire window)
    last_step_feat = x[:, -1, :]              # tensor for last time-step features
    outputs = layers.Dense(num_classes, activation='softmax')(last_step_feat)

    model = models.Model(inputs, outputs)
    return model

# Build the model and show summary
num_classes = len(encoder.classes_)
tcn_model = build_tcn_model(input_shape=(window_size, X_train.shape[2]), num_classes=num_classes,
                             num_filters=32, kernel_size=3, dilations=[1, 2, 4], dropout_rate=0.1)
tcn_model.summary()

## Model Training with Early Stopping

In [ ]:
# Compile the model
tcn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping callback
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = tcn_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,             # maximum epochs (we expect early stopping to halt before this if converged)
    batch_size=32,         # you can adjust batch size based on data size
    callbacks=[early_stop],
    verbose=1             # verbose=1 to print training progress
)

## Model Evaluation and Metrics

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# Predict class probabilities on the test set
y_prob = tcn_model.predict(X_test)
y_pred = y_prob.argmax(axis=1)          # predicted class indices

# Compute metrics
test_accuracy = accuracy_score(y_test, y_pred)
test_macro_f1 = f1_score(y_test, y_pred, average='macro')
test_macro_auc = roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro')

print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print(f"Test Macro F1-score: {test_macro_f1:.3f}")
print(f"Test Macro AUC: {test_macro_auc:.3f}")

## Hyperparameter Tuning

In [ ]:
!pip install keras-tuner -q

import keras_tuner as kt

# Define a model-building function for KerasTuner
def build_model(hp):
    # Hyperparameters to tune:
    filters = hp.Int('filters', min_value=16, max_value=64, step=16)            # e.g. 16, 32, 48, 64
    kernel = hp.Choice('kernel_size', values=[3, 5])                            # e.g. 3 or 5
    dropout = hp.Choice('dropout_rate', values=[0.0, 0.1, 0.2])
    # You could also tune number of dilation blocks, but for simplicity keep it fixed at [1,2,4]

    # Build TCN model with these hyperparameters
    model = build_tcn_model(input_shape=(window_size, X_train.shape[2]), num_classes=num_classes,
                             num_filters=filters, kernel_size=kernel, dilations=[1, 2, 4], dropout_rate=dropout)
    # Compile model with a potentially tunable learning rate
    lr = hp.Choice('learning_rate', values=[1e-3, 5e-4, 1e-4])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Set up Hyperband tuner (you can also use RandomSearch or BayesianOptimization)
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='tuner_dir',
    project_name='TCN_gait_tuning',
    overwrite=True
)

# Run the hyperparameter search
tuner.search(X_train, y_train,
             validation_data=(X_val, y_val),
             epochs=20,
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
            )

# Get the best hyperparameters and model
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters found:", best_hp.values)
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model on the test set
best_model.evaluate(X_test, y_test)